<a href="https://colab.research.google.com/github/yoonjong12/TransformerChatbot/blob/master/Train_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer Korean Chatbot
트랜스포머(Transformer)를 활용한 한국어 챗봇입니다.
* 학습 데이터는 '9회 투빅스 컨퍼런스 - 장애인의 용이한 정보접근을 위한 챗봇'을 준비하면서 수집한 장애인 관련 말뭉치들입니다.\
이 데이터는 저 혼자 전처리하고 수집한 것이 아니기 때문에 독단으로 공개하기 어려울 것 같습니다.\
다만, Test_Transformer.ipynb에서 챗봇을 직접 구동해보실 수 있습니다.

* 이 Colab 노트북 파일은 어떻게 트랜스포머 모델을 tensorflow2.0 프레임워크에 맞추어 학습을 시키면 되는지 pipeline을 보여드리고자 준비했습니다.

* 만약 Colab이 아닌 로컬에서 사용하고 싶으시다면 Colab에만 쓰이는 코드는 제외하시기 바랍니다.(명시는 해놓았으니 코드를 따라가주세요)

* 트랜스포머 모듈의 자세한 설명은 module > Transformer.py에 주석으로 했습니다. 참고해주세요

In [0]:
# Colab과 개인 Gdrive를 연동하기 위한 코드입니다.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Tensorflow2.0 환경을 Colab에서 사용하기 위한 코드입니다.
try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception as err:
    print(str(err))
    pass

# import library
from tensorflow import keras
import os
import sys
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text
from keras import metrics
from keras import backend
tf.random.set_seed(1234)

# GPU로 학습을 시키기
# GPU를 사용하기 위하여 Colab의 런타임 메뉴 > 런타임 유형 변경 > 하드웨어 가속기 에서 GPU로 설정하시기 바랍니다.  
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
    print('GPU device not found')
print(f'GPU at : {device}')

GPU at : /device:GPU:0


In [0]:
!git clone https://github.com/yoonjong12/TransformerChatbot.git

# 깃허브로 클론한 모듈을 임포트합니다
from TransformerChatbot.module.Transformer import *

fatal: destination path 'TransformerChatbot' already exists and is not an empty directory.


# Data Loading & Preprocessing
* 탄시리팀의 말뭉치를 로드하고 전처리합니다.
* Preprocess클래스는 제가 직접 구현하였으며, 간단하게 올바른 인자만 넣어주면 학습 준비를 쉽게 할 수 있습니다.
* 만약 데이터 배포가 가능해지거나, 다른 좋은 말뭉치가 있다면 수정하겠습니다.

In [0]:
# Corpus 
corpus = '/content/drive/My Drive/corpus_data.pickle'
df = pd.read_pickle(corpus)
df.head()

,Question,Answer,pre_Question,pre_Answer,Q_token,A_token,len_Q_token,len_A_token
0,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,"[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 만, 21,...","[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 초, 중등교...",25,65
1,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요?,장애인활동지원 홈페이지(www.ableservice.or.kr)에서 지정된 활동 지...,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요,장애인활동지원 홈페이지wwwableserviceorkr에서 지정된 활동 지원기관 및...,"[장애인, 활동, 지원, 교육, 기관, 과, 제, 공, 기관, 에, 대한, 정보, ...","[장애인, 활동, 지원, 홈페이지, wwwableserviceorkr, 에서, 지정...",19,21
2,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요?,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다.,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다,"[기초, 연금, 수급, 권, 을, 포기, 하면, 장애인, 연금, 기초, 급여, 를,...","[장애인, 연금, 법, 제, 6조, 제, 5, 항, 에, 따라, 기초, 연금, 수급...",15,22
3,만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,"장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며, 미리 신청하더라도 ...",만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며 미리 신청하더라도 3...,"[만, 19, 세, 중, 증장, 애인, 이, 2월, 에, 학교, 를, 졸업, 하고,...","[장애인, 연금, 법, 에, 따라, 2월, 중, 에는, 장애인, 연금, 을, 신청,...",25,24
4,장애인연금은 누가 받을 수 있나요?,"만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인(종전 1, 2급 및 3급...",장애인연금은 누가 받을 수 있나요,만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인종전 1 2급 및 3급 중...,"[장애인, 연금, 은, 누가, 받을, 수, 있나요]","[만, 18, 세, 이상, 의, 등록, 장애인, 중, 장애인, 연금, 법, 상, 중...",7,32


In [0]:
questions = df['Q_token']
answers = df['A_token']
MAX_LENGTH = 30
preprocess = Preprocess()

# 옵션1) 새로운 토크나이저 만들기
tokenizer = preprocess.buildTokenizer(questions, answers)
questions_seq, answers_seq = preprocess.tokenize_and_filter(tokenizer, MAX_LENGTH)
# 토크나이저를 load또는 save하기 위한 주소
tk_dir = '/content/drive/My Drive/tokenizer_data.json'
# 옵션2) 토크나이저 저장
preprocess.saveTokenizer(tk_dir, tokenizer)
# 옵션3) 기존 토크나이저 로드
# tokenizer = preprocess.loadTokenzier(tk_dir)
# preprocess.questions = questions
# preprocess.answers = answers
# questions_seq, answers_seq = preprocess.tokenize_and_filter(tokenizer, MAX_LENGTH)

print('Vocab size: {}'.format(preprocess.VOCAB_SIZE))
print('Number of samples: {}'.format(len(questions_seq)))

Vocab size: 57966
Number of samples: 1570


In [0]:
# dataset
# 입력 데이터와 출력 데이터, 배치사이즈만 인자로 넣어주면 자동으로 dataset을 만드는 함수입니다.
# Tensorflow의 dataset은 batch train을 용이하게 해줍니다.
BATCH_SIZE = 32
dataset = preprocess.buildDataset(questions_seq, answers_seq, BATCH_SIZE)
dataset

<PrefetchDataset shapes: ({inputs: (None, 30), dec_inputs: (None, 29)}, {outputs: (None, 29)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>

# Train
* 파라미터는 Transformer 논문과 일부 다를 수 있습니다.
* github에 업로드하기 위해서는 100mb 미만이여야 하기때문에 모델 파라미터 크기를 많이 낮추었습니다.
1. 각 파라미터를 설정해줍니다. 
2. model이 학습할 Transformer 모델입니다. 인자로 파라미터를 넣어주었습니다.
3. 학습률과 옵티마이저를 설정하고 compile로 학습할 준비를 마칩니다.
4. fit으로 학습을 시작합니다.

In [0]:
tf.keras.backend.clear_session()

# parameters
NUM_LAYERS = 2
D_MODEL = 32
NUM_HEADS = 4
UNITS = 64
DROPOUT = 0.2
VOCAB_SIZE = preprocess.VOCAB_SIZE

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=customLoss(MAX_LENGTH), metrics=[custom_accuracy(MAX_LENGTH)])

# 옵션1) 재학습을 위한 모델 로드
# MAX_LENGTH = 30
# custom_objects = {'PositionalEncoding':PositionalEncoding,
#                   'MultiHeadAttention':MultiHeadAttention,
#                   'CustomSchedule':CustomSchedule,
#                   'loss_function':customLoss(MAX_LENGTH),
#                   'accuracy':custom_accuracy(MAX_LENGTH), 
#                   'create_padding_mask':create_padding_mask,
#                   'backend':backend, 
#                   'tf':tf}
# model = keras.models.load_model(path, custom_objects=custom_objects)

# 옵션2) 모델 plot 시각화와 저장
# tf.keras.utils.plot_model(model, to_file='transformer.png', show_shapes=True)

In [0]:
steps = len(questions_seq) // BATCH_SIZE
EPOCHS = 150
print(f'MAX_LENGTH : {MAX_LENGTH}, EPOCHS : {EPOCHS}')
model.fit(dataset, epochs=EPOCHS, steps_per_epoch=steps)

# 옵션) verbose = 0 모드일 떄 최종 train acc 찍어보는 거
# test_scores = model.evaluate(dataset, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])

MAX_LENGTH : 30, EPOCHS : 150
Train for 49 steps
Epoch 1/150
49/49 [==============================] - 6s 122ms/step - loss: 6.5835 - accuracy: 4.3983e-05
Epoch 2/150
49/49 [==============================] - 6s 120ms/step - loss: 6.5841 - accuracy: 0.0012
Epoch 3/150
49/49 [==============================] - 6s 121ms/step - loss: 6.4192 - accuracy: 0.0054
Epoch 4/150
49/49 [==============================] - 6s 121ms/step - loss: 6.2089 - accuracy: 0.0128
Epoch 5/150
49/49 [==============================] - 6s 120ms/step - loss: 5.9902 - accuracy: 0.0203
Epoch 6/150
49/49 [==============================] - 6s 120ms/step - loss: 5.8755 - accuracy: 0.0265
Epoch 7/150
49/49 [==============================] - 6s 121ms/step - loss: 5.4759 - accuracy: 0.0326
Epoch 8/150
49/49 [==============================] - 6s 122ms/step - loss: 5.1412 - accuracy: 0.0344
Epoch 9/150
49/49 [==============================] - 6s 120ms/step - loss: 4.8436 - accuracy: 0.0345
Epoch 10/150
49/49 [==================

In [0]:
# 모델 저장
# 단순히 가중치만 저장하는 것이 아닌 모델 전체를 저장하기 때문에
# 다시 로드하여 학습을 이어서 할 수 있습니다.
model_path = '/content/drive/My Drive/my_keras_model.h5'
model.save(model_path)
del model

여기까지 모델을 학습하고 저장했습니다.\
다음으로 모델에 평가용 텍스트를 입력하여 출력받는 노트북을 확인해주세요
